# Unit 4

## Modules: The DSPy Building Blocks

## Introduction to DSPy Modules

Welcome to the fourth lesson in the **DSPy Programming** course\! This lesson builds on the previous one, where we learned about creating custom DSPy signatures. If signatures are the contracts that define the inputs and outputs of a language model task, then **modules** are the implementations that fulfill those contracts. They are the core building blocks you use to construct AI systems in DSPy.

Think of it like this: a signature says, "I need a function that takes a question and returns an answer," while a module says, "Here's how I'm going to transform that question into an answer." Modules encapsulate specific prompting techniques and reasoning strategies, letting you use sophisticated approaches without having to create them from scratch.

DSPy modules are inspired by neural network modules in PyTorch, but instead of working with tensors, they operate on natural language. You can compose DSPy modules to build complex language model applications, just as you would with PyTorch modules.

Each module in DSPy is designed to implement a specific prompting technique, like **chain-of-thought reasoning** or **tool use**. By combining these modules, you can create powerful AI systems that leverage the strengths of different approaches.

-----

## The Module Usage Pattern

Working with DSPy modules follows a consistent three-step pattern:

1.  **Declare the module with a signature:** This specifies the expected inputs and outputs.
2.  **Call the module with input arguments:** This provides the actual data for processing.
3.  **Access the outputs from the returned prediction:** This retrieves the results.

### Simple Example: `dspy.Predict`

The `dspy.Predict` module is the most basic module. Here's how the pattern works with it:

```python
# Step 1: Declare with a signature
classify = dspy.Predict('sentence -> sentiment: bool')

# Step 2: Call with input arguments
sentence = "it's a charming and often affecting journey."
response = classify(sentence=sentence)

# Step 3: Access the output
print(response.sentiment)
# Output: True
```

### More Complex Example: `dspy.ChainOfThought`

This same pattern applies to more sophisticated modules like `dspy.ChainOfThought`, which automatically adds a `reasoning` field to capture the model's step-by-step thinking.

```python
# Step 1: Declare with a signature
math = dspy.ChainOfThought("question -> answer: float")

# Step 2: Call with input argument
response = math(question="Two dice are tossed. What is the probability that the sum equals two?")

# Step 3: Access the outputs
print(f"Reasoning: {response.reasoning}")
print(f"Answer: {response.answer}")
```

You can also pass **configuration options** when declaring a module, such as specifying the number of completions to generate (`n=5`):

```python
# Step 1: Declare with a signature and configuration
classify = dspy.ChainOfThought('question -> answer', n=5)

# Step 2: Call with input argument
question = "What's something great about the ColBERT retrieval model?"
response = classify(question=question)

# Step 3: Access the outputs
for i, answer in enumerate(response.completions.answer):
    print(f"Completion {i+1}: {answer}")
```

This consistent three-step pattern makes DSPy code predictable and easy to understand.

-----

## Core Module Types

DSPy provides several built-in modules, each implementing a different prompting technique.

### Predict

The `Predict` module is the simplest and most fundamental. It directly implements a signature without adding extra fields or behavior, making it perfect for straightforward tasks.

**Example with a class-based signature:**

```python
from typing import Literal
import dspy

class Classify(dspy.Signature):
    """Classify sentiment of a given sentence."""
    sentence: str = dspy.InputField()
    sentiment: Literal['positive', 'negative', 'neutral'] = dspy.OutputField()
    confidence: float = dspy.OutputField()
    
classify = dspy.Predict(Classify)
result = classify(sentence="This book was super fun to read, though not the last chapter.")
print(f"Sentiment: {result.sentiment}")
print(f"Confidence: {result.confidence}")
```

### ChainOfThought

The `ChainOfThought` module encourages the language model to think step-by-step before providing an answer, which often leads to more accurate results for complex reasoning tasks. It automatically adds a `reasoning` field to capture this process.

**Example:**

```python
math = dspy.ChainOfThought("question -> answer: float")
result = math(question="Two dice are tossed. What is the probability that the sum equals two?")
print(f"Reasoning: {result.reasoning}")
print(f"Answer: {result.answer}")
```

### ProgramOfThought

The `ProgramOfThought` module takes reasoning a step further by encouraging the language model to generate and execute code to solve problems, which is particularly useful for mathematical or algorithmic tasks.

**Example:**

```python
example = dspy.ProgramOfThought("question -> answer")
question = "I have 10 candies, I want to distribute them between my 5 friends, how many candies I have to give to each friend?"
result = example(question=question)
print(f"Question: {question}")
print(f"Final Predicted Answer (after ProgramOfThought process): {result.answer}")
```

Behind the scenes, this module might generate and run code like `10 / 5` to get a precise answer.

### ReAct

The `ReAct` module implements the **Reasoning and Acting framework**, allowing the language model to use external tools to gather information or perform actions as part of its reasoning. This is great for tasks that need external knowledge or computation.

**Example:**

```python
def evaluate_math(expression: str) -> float:
    return dspy.PythonInterpreter({}).execute(expression)
    
def search_wikipedia(query: str) -> str:
    results = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')(query, k=3)
    return [x['text'] for x in results]
    
react = dspy.ReAct("question -> answer: float", tools=[evaluate_math, search_wikipedia])
pred = react(question="What is 9362158 divided by the year of birth of David Gregory of Kinnairdy castle?")
print(pred.answer)
```

### MultiChainComparison

The `MultiChainComparison` module generates multiple reasoning paths and then compares them to select the best answer. This can lead to more robust results, especially for questions where different reasoning approaches might lead to different answers.

**Example:**

```python
import dspy

class MultipleChoice(dspy.Signature):
    """Answer multiple choice questions by comparing options."""
    
    question = dspy.InputField()
    options = dspy.InputField()
    answer = dspy.OutputField(desc="The correct answer (A, B, C, or D)")
    reasoning = dspy.OutputField(desc="Explanation for the answer")
    
predictor = dspy.Predict(MultipleChoice)
mc_solver = dspy.MultiChainComparison(MultipleChoice, M=3)
question = "What color is the sky?"
options = {"A": "Red", "B": "Blue", "C": "Black", "D": "Yellow"}
completions = [predictor(question=question, options=options) for _ in range(3)]
    
result = mc_solver(completions=completions, question=question, options=options)
print(f"Selected Answer: {result.answer}")
print(f"Reasoning: {result.rationale}")
```

Each of these core modules is suited to different types of tasks.

-----

## Configuring Module Behavior

DSPy modules can be configured in various ways to control how they interact with the language model. You can specify parameters such as the number of completions (`n`), temperature, or maximum tokens.

**Example with `ChainOfThought`:**

```python
# Configure the module to generate multiple completions
classify = dspy.ChainOfThought('question -> answer', n=5)
question = "What's something great about the ColBERT retrieval model?"
response = classify(question=question)

# Access the multiple completions
for i, answer in enumerate(response.completions.answer):
    print(f"Completion {i+1}: {answer}")
    
# Access the selected completion
print(f"\nSelected answer: {response.answer}")
```

You can also set language model parameters, such as `temperature` and `max_tokens`, to fine-tune the output.

-----

## Composing Modules into Programs

One of DSPy's most powerful features is the ability to compose modules into larger programs by creating a custom class that inherits from `dspy.Module`. You define a `forward` method where you can use other DSPy modules to process inputs and generate outputs.

**Example of a custom `Hop` module for multi-hop retrieval:**

```python
import dspy

class Hop(dspy.Module):
    def __init__(self, num_docs=10, num_hops=4):
        self.num_docs, self.num_hops = num_docs, num_hops
        self.generate_query = dspy.ChainOfThought('claim, notes -> query')
        self.append_notes = dspy.ChainOfThought('claim, notes, context -> new_notes: list[str], titles: list[str]')
        
    def forward(self, claim: str) -> list[str]:
        notes = []
        titles = []
        for _ in range(self.num_hops):
            query = self.generate_query(claim=claim, notes=notes).query
            context = search(query, k=self.num_docs)
            prediction = self.append_notes(claim=claim, notes=notes, context=context)
            notes.extend(prediction.new_notes)
            titles.extend(prediction.titles)
        return dspy.Prediction(notes=notes, titles=list(set(titles)))
```

This modular approach makes it easy to experiment with different strategies and iterate on your designs.

-----

## Conclusion

This lesson covered the **DSPy modules**, the fundamental building blocks of DSPy programs. We explored the core module types, learned how to use and configure them, and saw how to compose them into larger, more sophisticated applications. By remembering the consistent three-step pattern—declare, call, and access—you can effectively use any DSPy module to create powerful AI systems.

## Building Your Second Sentiment Classifier

Now that you've learned about the three-step pattern for using DSPy modules, let's put it into practice! In this exercise, you'll create a simple sentiment classifier using the Predict module — the most basic building block in DSPy.

Your task is to complete the solution.py file by following these three steps:

Declare a Predict module with a signature that takes a sentence and returns a boolean sentiment.
Call this module with the provided example sentence.
Access and print the sentiment result from the response.
This hands-on exercise will help you understand the fundamental pattern that applies to all DSPy modules. Once you master this pattern, you'll be ready to work with more complex modules like ChainOfThought and ReAct in future exercises.

```python
import dspy
import os
from dspy.predict import Predict

# Configure a language model
lm = dspy.LM('openai/gpt-4o-mini', api_key=os.environ['OPENAI_API_KEY'], api_base=os.environ['OPENAI_BASE_URL'])
dspy.configure(lm=lm)

# Example sentence for sentiment analysis
sentence = "it's a charming and often affecting journey."

# TODO: Step 1 - Declare a Predict module with a signature that takes a sentence and returns a boolean sentiment

# TODO: Step 2 - Call the module with the sentence as input

# TODO: Step 3 - Access and print the sentiment from the response
print(f"Sentence: {sentence}")
print(f"Sentiment (True = positive, False = negative): ")

```

```python
import dspy
import os
from dspy.predict import Predict

# Configure a language model
lm = dspy.LM('openai/gpt-4o-mini', api_key=os.environ['OPENAI_API_KEY'], api_base=os.environ['OPENAI_BASE_URL'])
dspy.configure(lm=lm)

# Example sentence for sentiment analysis
sentence = "it's a charming and often affecting journey."

# Step 1: Declare a Predict module with a signature
classify = dspy.Predict('sentence -> sentiment: bool')

# Step 2: Call the module with the sentence as input
response = classify(sentence=sentence)

# Step 3: Access and print the sentiment from the response
print(f"Sentence: {sentence}")
print(f"Sentiment (True = positive, False = negative): {response.sentiment}")
```

## Step by Step Math Reasoning

Now that you've learned about the module usage pattern, let's put it into practice with the ChainOfThought module! In this exercise, you'll create a math problem solver that demonstrates how this module enhances basic prediction by adding step-by-step reasoning.

You'll follow the three-step pattern we just covered:

Declare a ChainOfThought module with a signature for solving math problems.
Call the module with a probability question about dice.
Access both the reasoning process and the final answer.
This exercise will help you see how ChainOfThought improves basic prediction by making the model's thinking process transparent. By examining the reasoning field, you'll gain insight into how the language model approaches complex problems step by step before arriving at a solution.

```python
import dspy
import os

# Configure a language model
lm = dspy.LM('openai/gpt-4o-mini', api_key=os.environ['OPENAI_API_KEY'], api_base=os.environ['OPENAI_BASE_URL'])
dspy.configure(lm=lm)

# TODO: Declare a ChainOfThought module with a signature that takes a question and returns an answer

# Define the math problem
question = "Two dice are tossed. What is the probability that the sum equals two?"

# TODO: Call the module with the problem

# TODO: Access and print both the reasoning and the answer

```

```python
import dspy
import os

# Configure a language model
lm = dspy.LM('openai/gpt-4o-mini', api_key=os.environ['OPENAI_API_KEY'], api_base=os.environ['OPENAI_BASE_URL'])
dspy.configure(lm=lm)

# Declare a ChainOfThought module with a signature that takes a question and returns an answer
math_solver = dspy.ChainOfThought("question -> answer")

# Define the math problem
question = "Two dice are tossed. What is the probability that the sum equals two?"

# Call the module with the problem
response = math_solver(question=question)

# Access and print both the reasoning and the answer
print(f"Question: {question}")
print("---")
print(f"Reasoning: {response.reasoning}")
print(f"Answer: {response.answer}")
```

## Code Powered Candy Distribution

After exploring how ChainOfThought helps with step-by-step reasoning, let's try another powerful module: ProgramOfThought! This module elevates problem-solving by actually generating and executing code to solve problems.

In this exercise, you'll create a candy distribution solver that uses code to calculate the answer. You'll follow the same three-step pattern:

Declare a ProgramOfThought module with a signature for solving distribution problems.
Call the module with a candy distribution question.
Access and print the final answer.
This exercise will show you how ProgramOfThought combines natural language understanding with the precision of code execution — perfect for mathematical problems where exact calculations matter. When you complete this exercise, you'll understand how DSPy can automatically generate code to solve problems without you having to write a single line of calculation code yourself!

```python
import dspy
import os

# Configure a language model
lm = dspy.LM('openai/gpt-4o-mini', api_key=os.environ['OPENAI_API_KEY'], api_base=os.environ['OPENAI_BASE_URL'])
dspy.configure(lm=lm)

# TODO: Declare the ProgramOfThought module with a signature that takes a question and returns an answer

# Define the candy distribution problem
question = "I have 15 candies, and I want to distribute them equally among 6 friends. How many candies will each friend get, and how many will I have left over?"

# TODO: Call the module with the problem

# TODO: Access and print the answer
```

```python
import dspy
import os

# Configure a language model
lm = dspy.LM('openai/gpt-4o-mini', api_key=os.environ['OPENAI_API_KEY'], api_base=os.environ['OPENAI_BASE_URL'])
dspy.configure(lm=lm)

# Declare the ProgramOfThought module with a signature that takes a question and returns an answer
solver = dspy.ProgramOfThought("question -> answer")

# Define the candy distribution problem
question = "I have 15 candies, and I want to distribute them equally among 6 friends. How many candies will each friend get, and how many will I have left over?"

# Call the module with the problem
response = solver(question=question)

# Access and print the answer
print(f"Question: {question}")
print(f"Final Predicted Answer: {response.answer}")
```

## Tools and Reasoning with ReAct

Now that you've seen how ProgramOfThought can generate code to solve problems, let's explore the ReAct module! This powerful module takes things a step further by combining reasoning with tool usage.

In this exercise, you'll build a calculator that can solve distribution problems by using a math evaluation tool. You'll follow our familiar three-step pattern:

Declare a ReAct module with a signature and configure it to use the provided math calculation tool.
Call the module with a question that requires both reasoning and calculation.
Access and display the answer.
This exercise shows how ReAct can dynamically decide when to use tools versus when to rely on its own reasoning. By completing this task, you'll understand how to build AI systems that can think and act — a key capability for solving complex real-world problems!

```python
import dspy
import os

# Configure a language model
lm = dspy.LM('openai/gpt-4o-mini', api_key=os.environ['OPENAI_API_KEY'], api_base=os.environ['OPENAI_BASE_URL'])
dspy.configure(lm=lm)

# Define a tool function for mathematical calculations
def calculate_math(expression: str) -> float:
    """Evaluates a mathematical expression and returns the result."""
    return dspy.PythonInterpreter({}).execute(expression)

# TODO: Declare the ReAct module with a signature and the calculation tool

# Define a question that requires calculation
question = "If I have 5 boxes with 12 items in each, and I need to distribute them equally among 8 people, how many items will each person get?"

# TODO: Call the module with the question

# TODO: Access and print the answer
```

```python
import dspy
import os

# Configure a language model
lm = dspy.LM('openai/gpt-4o-mini', api_key=os.environ['OPENAI_API_KEY'], api_base=os.environ['OPENAI_BASE_URL'])
dspy.configure(lm=lm)

# Define a tool function for mathematical calculations
def calculate_math(expression: str) -> float:
    """Evaluates a mathematical expression and returns the result."""
    return dspy.PythonInterpreter({}).execute(expression)

# Declare the ReAct module with a signature and the calculation tool
solver = dspy.ReAct("question -> answer: float", tools=[calculate_math])

# Define a question that requires calculation
question = "If I have 5 boxes with 12 items in each, and I need to distribute them equally among 8 people, how many items will each person get?"

# Call the module with the question
response = solver(question=question)

# Access and print the answer
print(f"Question: {question}")
print(f"Answer: {response.answer}")
```

## Comparing Paths for Better Answers

After working with ReAct to combine reasoning with tools, let's explore the MultiChainComparison module! This clever module helps improve answer quality by comparing multiple reasoning paths.

In this exercise, you'll build a multiple-choice question solver that:

Defines a signature for multiple-choice questions with fields for the question, options, answer, and reasoning
Creates multiple predictions using a basic predictor
Uses MultiChainComparison to select the best answer by comparing these different reasoning paths
You'll see how comparing multiple approaches can lead to more reliable answers than relying on a single prediction. This technique is especially valuable for questions where different reasoning strategies might lead to different conclusions.

By completing this exercise, you'll add another powerful technique to your DSPy toolkit — one that can significantly improve the reliability of your AI systems!

```python
import dspy
import os

# Configure a language model
lm = dspy.LM('openai/gpt-4o-mini', api_key=os.environ['OPENAI_API_KEY'], api_base=os.environ['OPENAI_BASE_URL'])
dspy.configure(lm=lm)

# TODO: Define a signature for multiple-choice questions with input fields for question and options,
# and output fields for answer and reasoning

# TODO: Create a basic predictor using the signature

# TODO: Create a MultiChainComparison module with M=3 (to compare 3 reasoning paths)

# Define a multiple-choice question
question = "Which planet is known as the 'Red Planet'?"
options = {
    "A": "Venus", 
    "B": "Mars", 
    "C": "Jupiter", 
    "D": "Saturn"
}

# TODO: Generate multiple predictions (at least 3) using the basic predictor

# TODO: Use MultiChainComparison to select the best answer based on the generated predictions

# TODO: Display the question, options, selected answer, and reasoning
```

```python
import dspy
import os
from dspy.predict import Predict
from dspy.retrieve import MultiChainComparison

# Configure a language model
lm = dspy.LM('openai/gpt-4o-mini', api_key=os.environ['OPENAI_API_KEY'], api_base=os.environ['OPENAI_BASE_URL'])
dspy.configure(lm=lm)

# Define a signature for multiple-choice questions
class MultipleChoice(dspy.Signature):
    """Answer multiple choice questions by comparing options."""
    question = dspy.InputField()
    options = dspy.InputField()
    answer = dspy.OutputField(desc="The correct answer (A, B, C, or D)")
    reasoning = dspy.OutputField(desc="Explanation for the answer")

# Create a basic predictor using the signature
predictor = dspy.Predict(MultipleChoice)

# Create a MultiChainComparison module with M=3
mc_solver = dspy.MultiChainComparison(MultipleChoice, M=3)

# Define a multiple-choice question
question = "Which planet is known as the 'Red Planet'?"
options = {
    "A": "Venus", 
    "B": "Mars", 
    "C": "Jupiter", 
    "D": "Saturn"
}

# Generate multiple predictions (at least 3)
completions = [predictor(question=question, options=options) for _ in range(3)]

# Use MultiChainComparison to select the best answer
result = mc_solver(completions=completions, question=question, options=options)

# Display the question, options, selected answer, and reasoning
print(f"Question: {question}")
print(f"Options: {options}")
print("---")
print(f"Selected Answer: {result.answer}")
print(f"Reasoning: {result.rationale}")
```